In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
#train_set
train_x = []
file = open("ocr/train-data.csv", "r")

for line in file:
    line = line.rstrip().split(",")
    train_x.append(line)

file.close()

train_x = np.array(train_x, dtype=float)

data = []
file = open("ocr/train-target.csv", "r")

for line in file:
    label = line.split()[0]
    data.append(label)

file.close()

train_label_ids = sorted(list(dict.fromkeys(data)))
train_n_labels = len(train_label_ids)
one_hot = []

for label in train_y:
    index = train_label_ids.index(label)
    vect = np.zeros(train_n_labels)
    vect[index] = 1
    one_hot.append(vect)

train_y = np.array(one_hot, dtype=float)

train_x_row, train_x_column = train_x.shape
train_y_row, train_y_column = train_y.shape

#test-set
test_x = []
file = open("ocr/test-data.csv", "r")

for line in file:
    line = line.rstrip().split(",")
    test_x.append(line)

file.close()

test_x = np.array(test_x, dtype=float)

data = []
file = open("ocr/test-target.csv", "r")

for line in file:
    label = line.split()[0]
    data.append(label)

file.close()

test_label_ids = sorted(list(dict.fromkeys(data)))
test_n_labels = len(test_label_ids)
one_hot = []

for label in test_y:
    index = test_label_ids.index(label)
    vect = np.zeros(test_n_labels)
    vect[index] = 1
    one_hot.append(vect)

test_y = np.array(one_hot, dtype=float)

test_x_row, test_x_column = test_x.shape
test_y_row, test_y_column = test_y.shape

In [14]:
x = tf.placeholder(tf.float32, [None, train_x_column])
y = tf.placeholder(tf.float32, [None, test_y_column])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [15]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', use_cudnn_on_gpu=False)

In [16]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])


x_image = tf.reshape(x, [-1, 16, 8, 1])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


W_fc1 = weight_variable([4 * 2 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 4*2*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


W_fc2 = weight_variable([1024, test_n_labels])
b_fc2 = bias_variable([test_n_labels])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_hat = tf.nn.softmax(y_conv)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_hat), reduction_indices=[1]))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [17]:
predicted_y = tf.argmax(y_hat, 1)
real_y = tf.argmax(y, 1)
correct_prediction = tf.equal(predicted_y, real_y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

batch_size = 50
epochs = int(train_x_row/batch_size+1)

for i in range(epochs):
    start = i*batch_size
    end = (i+1)*batch_size
    
    if (((i%100)==0) | (i==int(train_x_row/batch_size))):
        train_accuracy = accuracy.eval(feed_dict={x: train_x[start:end], y: train_y[start:end], keep_prob: 1.0})
        
        print('Epochs {}: training accuracy {}'.format(i, train_accuracy))
    
    train_step.run(feed_dict={x: train_x[start:end], y: train_y[start:end], keep_prob: 0.5})

accuracy_values = []
predicted_targets = []

batch_size = 100
epochs = int(test_x_row/batch_size+1)

for j in range(epochs):
    start = j*batch_size
    end = (j+1)*batch_size
    acc, pred_y = session.run([accuracy, predicted_y], feed_dict={x: test_x[start:end], y: test_y[start:end], keep_prob: 1.0})
    
    
    accuracy_values.append(acc)
    predicted_targets.extend(pred_y)

session.close()
    
print('\nTest accuracy {}'.format(np.mean(accuracy_values)))

Epochs 0: training accuracy 0.019999999552965164
Epochs 100: training accuracy 0.46000000834465027
Epochs 200: training accuracy 0.4000000059604645
Epochs 300: training accuracy 0.6200000047683716
Epochs 400: training accuracy 0.6399999856948853
Epochs 500: training accuracy 0.6600000262260437
Epochs 600: training accuracy 0.699999988079071
Epochs 700: training accuracy 0.7599999904632568
Epochs 800: training accuracy 0.7799999713897705
Epochs 834: training accuracy 0.761904776096344

Test accuracy 0.753966212272644


In [21]:
file = open("test-pred.csv", "w")

for prediction in predicted_targets:
    line = test_label_ids[prediction] + "\n"
    file.write(line)

file.close()